In [1]:
import pandas as pd
import numpy as np
import os
from sodapy import Socrata

%matplotlib inline

In [2]:
#get relevant collisions & speeds data from 2018

In [3]:
query = """
select *

where
date<'2019-01-01T00:00:00.000'
and
date>='2018-01-01T00:00:00.000'

and
borough='MANHATTAN'



limit
    50000000000
"""

In [4]:
#SQL querying collisions data from Socrata
client = Socrata("data.cityofnewyork.us", None)
results = client.get("qiz3-axqb", query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [5]:
query = """
select *

where
data_as_of<'2019-01-01T00:00:00.000'
and
data_as_of>='2018-01-01T00:00:00.000'
and
borough='Manhattan'
and
speed is not null
and
link_points is not null


limit
    50000000000
"""

In [6]:
#now for speeds data from Socrata again
client = Socrata("data.cityofnewyork.us", None)
#results = client.get("i4gi-tjb9", where="data_as_of='2017-10-19T06:58:30.000'",limit=2000000)
results = client.get("i4gi-tjb9", query=query)

# Convert to pandas DataFrame
results_df2 = pd.DataFrame.from_records(results)

In [7]:
df_speeds=results_df2.copy()
df_coll=results_df.copy()

In [10]:
#converting everything to proper date time format

In [11]:
df_coll['datetime']=pd.to_datetime(df_coll['date'].astype(str)+' '+df_coll['time'].astype(str))
df_speeds['datetime']=pd.to_datetime(df_speeds['data_as_of'])

In [12]:
df_coll['hour']=df_coll['datetime'].dt.hour
df_speeds['hour']=df_speeds['datetime'].dt.hour
df_speeds['speed']=np.array(df_speeds['speed'].values,dtype=float)
df_speeds=df_speeds.loc[df_speeds['speed']>0]

In [13]:
df_speeds['hm']=df_speeds['hour']*60+df_speeds['datetime'].dt.minute
df_coll['hm']=df_coll['hour']*60+df_coll['datetime'].dt.minute
df_coll['dateonly']=df_coll['datetime'].dt.date
df_speeds['dateonly']=df_speeds['datetime'].dt.date
df_speeds['weekday']=df_speeds['datetime'].dt.dayofweek
df_coll['weekday']=df_coll['datetime'].dt.dayofweek


In [24]:
df_coll.to_csv('df_coll.csv',index=False)

In [14]:
df_coll.columns

Index(['borough', 'contributing_factor_vehicle_1',
       'contributing_factor_vehicle_2', 'contributing_factor_vehicle_3',
       'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5',
       'cross_street_name', 'date', 'latitude', 'location', 'longitude',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_persons_injured', 'number_of_persons_killed',
       'off_street_name', 'on_street_name', 'time', 'unique_key',
       'vehicle_type_code1', 'vehicle_type_code2', 'vehicle_type_code_3',
       'vehicle_type_code_4', 'vehicle_type_code_5', 'zip_code', 'datetime',
       'hour', 'hm', 'dateonly', 'weekday'],
      dtype='object')

In [15]:
#Grouping columns to get rate of collisions per hour

grouped_df=df_coll.groupby(['hour','dateonly'])
df_coll_rate=pd.DataFrame(grouped_df.size().reset_index(name = "Coll_rate"))

In [17]:
#merging the collisions and speed datasets, grouped by hour and date

In [18]:
df_speeds_coll = pd.merge(df_speeds, df_coll_rate,  how='left', left_on=['hour','dateonly'], right_on = ['hour','dateonly'])

In [19]:
data=df_speeds_coll[['hm','id','weekday','Coll_rate','hour','dateonly','link_points']]
data['speed']=df_speeds_coll['speed']
data2=data.dropna()
features=data2[['hm','id','weekday','Coll_rate']]
labels=data2['speed']

C:\Users\Skanda\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
data2.head()

,hm,id,weekday,Coll_rate,hour,dateonly,link_points,speed
0,2,448,0,3.0,0,2018-01-01,"40.77149,-73.99423 40.7719,-73.99401 40.77481,...",39.76
1,2,330,0,3.0,0,2018-01-01,"40.75719,-73.99724 40.76017,-74.00382 40.76185...",41.01
2,2,364,0,3.0,0,2018-01-01,"40.745726,-73.97359 40.745616,-73.97305 40.745...",36.03
3,2,119,0,3.0,0,2018-01-01,"40.70631,-74.01501 40.705380,-74.01528 40.7049...",37.28
4,2,124,0,3.0,0,2018-01-01,"40.68036,-74.00441001 40.6822,-74.0057201 40.6...",36.66


In [23]:
data3=data2.groupby(['hour','dateonly','link_points','id']).mean().reset_index()
data3.to_csv('data3.csv',index=False)